In [7]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd

In [213]:
%run ./ml_model_init.ipynb

In [26]:
#
cs_data= data[(data['minutes_5'] >= 300) & (data['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]

cs_data_tar = data_tar[(data_tar['minutes_5'] >= 300) & (data_tar['position'] != 'FWD')][['position', 'minutes_3', 'minutes_5', 'team_h_difficulty', 'team_a_difficulty', 'was_home', 'clean_sheets', 'clean_sheets_3', 'clean_sheets_5',
                                                                         'expected_goals_conceded_3', 'expected_goals_conceded_5', 'ict_index_3',  'ict_index_5', 'influence_3', 'influence_5', 'creativity_3', 'creativity_5',
                                                                         'threat_3', 'threat_5', 'whh', 'whd', 'wha']]
# # players


In [147]:

cs = cs_data['clean_sheets']
feats = cs_data.drop('clean_sheets', axis=1)
feats_train, feats_test, cs_train, cs_test = train_test_split(feats, cs , test_size=0.2, random_state=42)

## Logistic Regression


In [203]:
preds = Logistic_regression(feats_train, feats_test, cs_train, cs_test)

Class distribution: {0: 1196, 1: 416}


In [217]:
print(preds['class_report'])

              precision    recall  f1-score   support

           0       0.78      0.54      0.64      1196
           1       0.30      0.56      0.39       416

    accuracy                           0.55      1612
   macro avg       0.54      0.55      0.51      1612
weighted avg       0.65      0.55      0.57      1612



In [205]:
tn, fp, fn, tp = preds['conf_mat'].ravel()
(tn, fp, fn, tp)

(649, 547, 184, 232)

## RandomForestClassifier


In [214]:
rf_preds = Random_Forest_Classifier(feats_train, feats_test, cs_train, cs_test)

Class distribution: {0: 1196, 1: 416}


In [215]:
print(rf_preds['class_report'])

              precision    recall  f1-score   support

           0       0.80      0.95      0.87      1196
           1       0.69      0.33      0.45       416

    accuracy                           0.79      1612
   macro avg       0.74      0.64      0.66      1612
weighted avg       0.77      0.79      0.76      1612



### Hyperparameter Tuning


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create a RandomForestClassifier model
rf = RandomForestClassifier(random_state=42)


## Baseline Model


#### Linear Model


In [13]:

lin_reg = Linear_regression(feats_train, feats_test, cs_train, cs_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = pd.DataFrame({"lin_reg": [lin_reg['train_MAE'], lin_reg['test_MAE'], lin_reg['train_RMSE'], lin_reg['test_RMSE'], lin_reg['cv_rmse'], lin_reg['R2_train'], lin_reg['R2_test']]},
                                                        index=(['train_MAE','test_MAE', 'train_RMSE', 'test_RMSE', 'cv_rmse', 'R2_train', 'R2_test']))

evaluation_stats

Training set RMSE: 0.42562893209867264
Test set RMSE: 0.43272260305042265
Training set R2: 0.017779491607455178
Test set R2: 0.02203181524839637


,lin_reg
train_MAE,0.362324
test_MAE,0.368142
train_RMSE,0.425629
test_RMSE,0.432723
cv_rmse,0.426978
R2_train,0.017779
R2_test,0.022032


#### DecisionTree Model


In [14]:
dt_reg = DecisionTreeRegression(feats_train, feats_test, cs_train, cs_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(dt_reg = [dt_reg['train_MAE'], dt_reg['test_MAE'], dt_reg['train_RMSE'], dt_reg['test_RMSE'], dt_reg['cv_rmse'], dt_reg['R2_train'], dt_reg['R2_test']])

evaluation_stats



,lin_reg,dt_reg
train_MAE,0.362324,4.376841e-17
test_MAE,0.368142,2.686104e-01
train_RMSE,0.425629,8.232589e-17
test_RMSE,0.432723,5.182764e-01
cv_rmse,0.426978,5.135181e-01
R2_train,0.017779,1.000000e+00
R2_test,0.022032,-4.029055e-01


#### RandomForest Model


In [15]:
hyperparameters = {"criterion": 'friedman_mse', "max_depth": 8, "max_features": 'sqrt', "n_estimators": 20}
rf_reg = RandomForestRegression(feats_train, feats_test, cs_train, cs_test, hyperparameters)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(rf_reg = [rf_reg['train_MAE'], rf_reg['test_MAE'], rf_reg['train_RMSE'], rf_reg['test_RMSE'], rf_reg['cv_rmse'], rf_reg['R2_train'], rf_reg['R2_test']])

evaluation_stats

,lin_reg,dt_reg,rf_reg
train_MAE,0.362324,4.376841e-17,0.310377
test_MAE,0.368142,2.686104e-01,0.349829
train_RMSE,0.425629,8.232589e-17,0.375623
test_RMSE,0.432723,5.182764e-01,0.417498
cv_rmse,0.426978,5.135181e-01,0.411198
R2_train,0.017779,1.000000e+00,0.235019
R2_test,0.022032,-4.029055e-01,0.089639


#### XgBoost Model


In [16]:
hyperparameters = {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 150}
xgb_reg = XGBoostRegression(feats_train, feats_test, cs_train, cs_test, hyperparameters)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(xgb_reg = [xgb_reg['train_MAE'], xgb_reg['test_MAE'], xgb_reg['train_RMSE'], xgb_reg['test_RMSE'], xgb_reg['cv_rmse'], xgb_reg['R2_train'], xgb_reg['R2_test']])
evaluation_stats

,lin_reg,dt_reg,rf_reg,xgb_reg
train_MAE,0.362324,4.376841e-17,0.310377,0.337162
test_MAE,0.368142,2.686104e-01,0.349829,0.356820
train_RMSE,0.425629,8.232589e-17,0.375623,0.397916
test_RMSE,0.432723,5.182764e-01,0.417498,0.419715
cv_rmse,0.426978,5.135181e-01,0.411198,0.414433
R2_train,0.017779,1.000000e+00,0.235019,0.141521
R2_test,0.022032,-4.029055e-01,0.089639,0.079943


# Feature engineering


### VarianceThreshold


In [17]:
# # Split into numerical and categorical columns
# bool_cols = feats_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
# categorical_cols = feats_train.select_dtypes(include=['object', 'category']).columns.tolist()

# # Preprocessing for numerical features
# numerical_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())
# ])

# # Preprocessing for categorical features
# categorical_transformer = Pipeline(steps=[
#     ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
# ])

# # Combine preprocessors in a column transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numerical_transformer, bool_cols),
#         ('cat', categorical_transformer, categorical_cols),
#     ]
# )

# # Scale and remove low-variance features
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     # ('var_threshold', VarianceThreshold(threshold=0.1))
# ])

# # Apply the pipeline to the training data
# pipeline.fit(feats_train)
# preprocessed_train = pipeline.transform(feats_train)
# preprocessed_test = pipeline.transform(feats_test)


# # # VT_scaler = StandardScaler()
# # train_scaled = pd.DataFrame(VT_scaler.fit_transform(feats_train), columns=feats_train.columns)
# # test_scaled = pd.DataFrame(VT_scaler.transform(feats_test), columns=feats_test.columns)

# # # selector = VarianceThreshold(threshold = 0.1)
# # # selector.fit_transform(df_for_train_scaled)

# # # for_threshold_columns = df_for_train_scaled.columns[selector.get_support()]

# # # for_threshold_train = df_for_train_scaled[for_threshold_columns]
# # # for_threshold_test = df_for_test_scaled[for_threshold_columns]

# # # for_threshold_train.shape, for_threshold_test.shape


# columns=feats_train.columns
# train_scaled = pd.DataFrame(preprocessed_train)
# test_scaled = pd.DataFrame(preprocessed_test)

# # # selector = VarianceThreshold(threshold = 0.1)
# # # selector.fit_transform(train_scaled)

# # # for_threshold_columns = train_scaled.columns[selector.get_support()]

# # # for_threshold_train = train_scaled[for_threshold_columns]
# # # for_threshold_test = test_scaled[for_threshold_columns]

# # # for_threshold_train.shape, for_threshold_test.shape
# # preprocessed_train
# # # print(preprocessed_train.shape, preprocessed_test.shape)
# train_scaled

In [18]:
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import pandas as pd

# Feature preprocessing pipeline
# Split into numerical and categorical columns
bool_cols = feats_train.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = feats_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
col_trans = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, bool_cols),
        ('cat', categorical_transformer, categorical_cols),
    ]
)

# Scale and remove low-variance features
preprocessor = Pipeline(steps=[
    ('col_trans', col_trans),
    ('var_threshold', VarianceThreshold(threshold=0.1))
])




#### Model(VT)


In [ ]:
def scores(model, feats_train, feats_test, cs_train, cs_test):
    # fit the transofrmer on the train data
    model.fit(feats_train, cs_train)

    support = model.regressor_.named_steps['preprocessor'].named_steps['var_threshold'].get_support()
    # print(f"Selected features: {feats_train.columns[support]}")
    print(len(support), len(feats_train.columns))

    # With the model fitted, we can predict the total_points given the feature_train and feature_test set
    pred_train = model.predict(feats_train)
    pred_test = model.predict(feats_test)

    # Evaluate the performance of the model on both sets using the mean absolute error
    train_MAE = mean_absolute_error(cs_train, pred_train)
    test_MAE = mean_absolute_error(cs_test, pred_test)

    # Evaluate the performance of the model on both sets using the mean square error
    train_MSE = mean_squared_error(cs_train, pred_train)
    test_MSE = mean_squared_error(cs_test, pred_test)

    # Evaluate the performance of the model on both sets using the root mean square error
    train_RMSE = mean_squared_error(cs_train, pred_train, squared=False)
    test_RMSE = mean_squared_error(cs_test, pred_test, squared=False)

    # Get the score of the model or the coeeficient of determination i.e how much of the target value can be explained by the model.
    # In this case, 0.6 implies that 60% of the variations in the target value can be explained by the model and 40% is not explainable
    R2_train = model.score(feats_train, cs_train)
    R2_test = model.score(feats_test, cs_test)

    # If the test error significantly differs from the train error, then there is either overfitting or underfitting
    # RMSE, just like the squared loss function that it derives from, effectively penalizes larger errors more severely.
    print('Training set RMSE: {}'.format(train_RMSE))
    print('Test set RMSE: {}'.format(test_RMSE))

    print('Training set R2: {}'.format(R2_train))
    print('Test set R2: {}'.format(R2_test))

    # Carry out cross validation of the model.
    # The evaluation method is the root mean square error
    # The method expects a utility function (greater is better) and so the scoring function is the opposite of the the RMSE. Hence the -ve
    tree_rmses = -cross_val_score(model, feats_train, cs_train,
                                    scoring="neg_root_mean_squared_error", cv=10)

    return {'train_MAE': train_MAE, 'test_MAE': test_MAE, 'train_MSE': train_MSE, 'test_MSE': test_MSE, 'train_RMSE': train_RMSE, 'test_RMSE': test_RMSE, 'cv_rmse': tree_rmses.mean(), 'R2_train': R2_train, 'R2_test': R2_test}


##### Linear Model


In [ ]:

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# TransformedTargetRegressor for target scaling
model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)

# Model predictions
VarT_lin_reg = scores(model, feats_train, feats_test, cs_train, cs_test)
# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_lin_reg =  [VarT_lin_reg['train_MAE'], VarT_lin_reg['test_MAE'],  VarT_lin_reg['train_RMSE'], VarT_lin_reg['test_RMSE'], VarT_lin_reg['cv_rmse'], VarT_lin_reg['R2_train'], VarT_lin_reg['R2_test']])

evaluation_stats

22 21
Training set RMSE: 0.42562893209867264
Test set RMSE: 0.4327226030504226
Training set R2: 0.017779491607455178
Test set R2: 0.022031815248396702


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg
train_MAE,0.362324,4.376841e-17,0.310377,0.337162,0.362324
test_MAE,0.368142,2.686104e-01,0.349829,0.356820,0.368142
train_RMSE,0.425629,8.232589e-17,0.375623,0.397916,0.425629
test_RMSE,0.432723,5.182764e-01,0.417498,0.419715,0.432723
cv_rmse,0.426978,5.135181e-01,0.411198,0.414433,0.426978
R2_train,0.017779,1.000000e+00,0.235019,0.141521,0.017779
R2_test,0.022032,-4.029055e-01,0.089639,0.079943,0.022032


##### DecisionTree Model


In [ ]:
# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])

# TransformedTargetRegressor for target scaling
dt_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)




VarT_dt_reg = scores(dt_model, feats_train, feats_test, cs_train, cs_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_dt_reg = [VarT_dt_reg['train_MAE'], VarT_dt_reg['test_MAE'], VarT_dt_reg['train_RMSE'], VarT_dt_reg['test_RMSE'], VarT_dt_reg['cv_rmse'], VarT_dt_reg['R2_train'], VarT_dt_reg['R2_test']])

evaluation_stats



22 21
Training set RMSE: 8.128678040360027e-17
Test set RMSE: 0.5030921754238529
Training set R2: 1.0
Test set R2: -0.32190635451505023


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg
train_MAE,0.362324,4.376841e-17,0.310377,0.337162,0.362324,4.282572e-17
test_MAE,0.368142,2.686104e-01,0.349829,0.356820,0.368142,2.531017e-01
train_RMSE,0.425629,8.232589e-17,0.375623,0.397916,0.425629,8.128678e-17
test_RMSE,0.432723,5.182764e-01,0.417498,0.419715,0.432723,5.030922e-01
cv_rmse,0.426978,5.135181e-01,0.411198,0.414433,0.426978,5.113866e-01
R2_train,0.017779,1.000000e+00,0.235019,0.141521,0.017779,1.000000e+00
R2_test,0.022032,-4.029055e-01,0.089639,0.079943,0.022032,-3.219064e-01


##### RandomForest Model


In [ ]:
hyperparameters = {"criterion": 'friedman_mse', "max_depth": 8, "max_features": 'sqrt', "n_estimators": 20}

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(
                        n_estimators=hyperparameters['n_estimators'],
                        max_depth=hyperparameters['max_depth'],
                        criterion=hyperparameters['criterion'], random_state=18
                        ))
])

# TransformedTargetRegressor for target scaling
rf_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)
# Store the model evaluation details in a DataFrame
VarT_rf_reg = scores(rf_model, feats_train, feats_test, cs_train, cs_test)

# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_rf_reg = [VarT_rf_reg['train_MAE'], VarT_rf_reg['test_MAE'], VarT_rf_reg['train_RMSE'], VarT_rf_reg['test_RMSE'], VarT_rf_reg['cv_rmse'], VarT_rf_reg['R2_train'], VarT_rf_reg['R2_test']])
evaluation_stats

22 21
Training set RMSE: 0.3755453557478455
Test set RMSE: 0.417084134191548
Training set R2: 0.23533448268702084
Test set R2: 0.09144148277426611


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg,VarT_rf_reg
train_MAE,0.362324,4.376841e-17,0.310377,0.337162,0.362324,4.282572e-17,0.310202
test_MAE,0.368142,2.686104e-01,0.349829,0.356820,0.368142,2.531017e-01,0.349486
train_RMSE,0.425629,8.232589e-17,0.375623,0.397916,0.425629,8.128678e-17,0.375545
test_RMSE,0.432723,5.182764e-01,0.417498,0.419715,0.432723,5.030922e-01,0.417084
cv_rmse,0.426978,5.135181e-01,0.411198,0.414433,0.426978,5.113866e-01,0.411220
R2_train,0.017779,1.000000e+00,0.235019,0.141521,0.017779,1.000000e+00,0.235334
R2_test,0.022032,-4.029055e-01,0.089639,0.079943,0.022032,-3.219064e-01,0.091441


##### XgBoost Model


In [ ]:
hyperparameters = {'learning_rate': 0.02, 'max_depth': 4, 'n_estimators': 150}

# Full pipeline including the regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb(learning_rate=hyperparameters["learning_rate"],
                    n_estimators=hyperparameters["n_estimators"],
                    max_depth=hyperparameters["max_depth"],
                    eval_metric='rmsle'))
])

# TransformedTargetRegressor for target scaling
rf_model = TransformedTargetRegressor(
    regressor=pipeline,
    transformer=StandardScaler()
)

VarT_xgb_reg = scores(rf_model, feats_train, feats_test, cs_train, cs_test)



# Store the model evaluation details in a DataFrame
evaluation_stats = evaluation_stats.assign(VarT_xgb_reg = [VarT_xgb_reg['train_MAE'], VarT_xgb_reg['test_MAE'], VarT_xgb_reg['train_RMSE'], VarT_xgb_reg['test_RMSE'], VarT_xgb_reg['cv_rmse'], VarT_xgb_reg['R2_train'], VarT_xgb_reg['R2_test']])
evaluation_stats

22 21
Training set RMSE: 0.3979160215859029
Test set RMSE: 0.41971509334827695
Training set R2: 0.14152121625756642
Test set R2: 0.07994298934476596


,lin_reg,dt_reg,rf_reg,xgb_reg,VarT_lin_reg,VarT_dt_reg,VarT_rf_reg,VarT_xgb_reg
train_MAE,0.362324,4.376841e-17,0.310377,0.337162,0.362324,4.282572e-17,0.310202,0.337162
test_MAE,0.368142,2.686104e-01,0.349829,0.356820,0.368142,2.531017e-01,0.349486,0.356820
train_RMSE,0.425629,8.232589e-17,0.375623,0.397916,0.425629,8.128678e-17,0.375545,0.397916
test_RMSE,0.432723,5.182764e-01,0.417498,0.419715,0.432723,5.030922e-01,0.417084,0.419715
cv_rmse,0.426978,5.135181e-01,0.411198,0.414433,0.426978,5.113866e-01,0.411220,0.414433
R2_train,0.017779,1.000000e+00,0.235019,0.141521,0.017779,1.000000e+00,0.235334,0.141521
R2_test,0.022032,-4.029055e-01,0.089639,0.079943,0.022032,-3.219064e-01,0.091441,0.079943
